In [79]:
import numpy as np
import json
from pathlib import Path
import os
import requests
from datetime import datetime
import datetime as dtt

def get_returns(candle):
    rendements = []
    for j in range(len(candle) - 1):
        j1 = candle[j]
        j2 = candle[j + 1]
        cash_flow = 0
        if "cash_flow" in j2:
            cash_flow = j2["cash_flow"]
        r = (j2["close"] - j1["close"] + cash_flow)/j1["close"]
        rendements.append(r)
    return np.array(rendements)
    
def get_vol(candle):
    closes = np.array([x["open"] for x in candles])
    return get_returns(candles).var() * 100
    
def get_volatility(id, start, end):
    param = {"ratio": [18], "asset": [id], "start_date": start, "end_date": end}
    s = requests.Session()
    s.auth = ('epita_user_6', 'dolphin39833')
    s.body = param
    r = s.post("https://dolphin.jump-technology.com:3389/api/v1/ratio/invoke", data = json.dumps(param), verify=False)
    r = r.json()
    print(r)
    
    
def pull_data():
    s = requests.Session()
    s.auth = ('epita_user_6', 'dolphin39833')
    r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset", verify=False)
    data = r.json()
    ids = [a["ASSET_DATABASE_ID"] for a in data]
    for i in ids:
        print(i["value"])
        p = Path("data/" + i["value"])
        if not p.exists():
            f = open("data/" + i["value"], "w")
            r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset/" + i["value"] + "/quote", verify=False)
            candles = r.json()
            f.write(json.dumps(candles))
            f.close()
    
    
def dateInPeriod(date, start, end):
    #start = datetime.strptime(start, "%Y-%m-%d")
    #end = datetime.strptime(end, "%Y-%m-%d")
    if date >= start and date <= end:
        return True
    return False
    
    
def cleanData(vector, start, end):
    temp = []
    for e in vector:
        e[0] = datetime.strptime(e[0], "%Y-%m-%d")
        if (dateInPeriod(e[0], start, end)):
            temp.append(e)
    
    cleaned = []
    prev = temp[0][0]
    s = start
    while s != prev:
        cleaned.append(temp[0][1])
        s = s + dtt.timedelta(days=1)
    cleaned.append(temp[0][1])

    for e in temp[1:]:
        while e[0] != prev + dtt.timedelta(days=1):
            cleaned.append(e[1])
            prev = prev + dtt.timedelta(days=1)
        cleaned.append(e[1])
        prev = e[0]
    while prev < end:
        cleaned.append(temp[-1][1])
        prev = prev + dtt.timedelta(days=1)
    return cleaned
    
def getMatrix(date_start, date_end):
    mat = []
    for root, dirs, files in os.walk("data/"):
        for f in files:
            if not f.startswith("."):
                fi = open(os.path.join(root, f))
                v = fi.readlines()[0]
                v = json.loads(v)
                ds = datetime.strptime(v[0]["date"], "%Y-%m-%d")
                de = datetime.strptime(v[-1]["date"], "%Y-%m-%d")
                if ds <= date_start and de >= date_end:
                    v = cleanData([[x["date"], float(x["close"])] for x in v], date_start, date_end)
                    v = np.array(v)
                    mat.append(v)
                #else:
                #    print("Can't take file ", f, " because starts at ", ds, " and end at ", de)
                fi.close()
               
    return np.array(mat)


#pull_data()
    
#def volatility(candle):
#def sharpe(candle):

In [47]:
import requests

s = requests.Session()
s.auth = ('epita_user_6', 'dolphin39833')
r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset", verify=False)
data = r.json()
ids = [a["ASSET_DATABASE_ID"] for a in data]
i = ids[0]
r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset/" + i["value"] + "/quote", verify=False)
candles = r.json()

/Users/Antoine/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/Antoine/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'value': '315', 'type': 'int32'}


In [99]:
get_returns(candles).mean()
print(get_vol(candles))
get_volatility(i["value"], candles[0]["date"], candles[-1]["date"])

0.0330155999393


/Users/Antoine/anaconda3/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'315': {'18': {'value': '0,288442914506', 'type': 'percent'}}}


In [39]:
np.sqrt(np.array([x["close"] for x in candles]).var())

8.4319540625874421

In [91]:
returns = get_returns(candles)
np.sqrt(np.abs(np.log(np.var(returns))))

2.8312444772164156

In [80]:
get_returns(candles)

array([ 0.        , -0.00763359, -0.01538462, ..., -0.00306686,
       -0.00041017,  0.01415675])

In [88]:
mat = getMatrix(datetime.strptime("2007-01-03", "%Y-%m-%d"), datetime.strptime("2017-06-30", "%Y-%m-%d"))
np.cov(mat)

array([[ 577.21960903,  198.83898832,  118.19372171, ...,   -5.37746714,
         171.6893691 ,   24.71168031],
       [ 198.83898832,  218.06453665,    2.39551423, ...,   -9.11176311,
          79.54541199,   12.7653513 ],
       [ 118.19372171,    2.39551423,  144.9466318 , ...,   32.60628136,
          19.13360553,   -1.34621958],
       ..., 
       [  -5.37746714,   -9.11176311,   32.60628136, ...,   11.67917023,
          -6.87981588,    0.87903832],
       [ 171.6893691 ,   79.54541199,   19.13360553, ...,   -6.87981588,
          60.07476948,    7.73301196],
       [  24.71168031,   12.7653513 ,   -1.34621958, ...,    0.87903832,
           7.73301196,   57.8102725 ]])

In [89]:
print(mat.shape)

(416, 3832)


In [39]:
datetime.strptime("2007-01-02", "%Y-%m-%d") + dtt.timedelta(days=1)

datetime.datetime(2007, 1, 3, 0, 0)

In [93]:
np.cov(mat)

array([[ 577.21960903,  198.83898832,  118.19372171, ...,   -5.37746714,
         171.6893691 ,   24.71168031],
       [ 198.83898832,  218.06453665,    2.39551423, ...,   -9.11176311,
          79.54541199,   12.7653513 ],
       [ 118.19372171,    2.39551423,  144.9466318 , ...,   32.60628136,
          19.13360553,   -1.34621958],
       ..., 
       [  -5.37746714,   -9.11176311,   32.60628136, ...,   11.67917023,
          -6.87981588,    0.87903832],
       [ 171.6893691 ,   79.54541199,   19.13360553, ...,   -6.87981588,
          60.07476948,    7.73301196],
       [  24.71168031,   12.7653513 ,   -1.34621958, ...,    0.87903832,
           7.73301196,   57.8102725 ]])